In [4]:
import pandas as pd
import os
import gc
import lightgbm as lgb
import xgboost as xgb
from sklearn.linear_model import SGDRegressor, LinearRegression, Ridge
from sklearn.preprocessing import MinMaxScaler
import math
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, log_loss
import matplotlib.pyplot as plt
import time
import warnings
warnings.filterwarnings('ignore')

In [6]:
train = pd.read_csv('train.csv')
testA = pd.read_csv('testA.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

In [5]:
train.head()

NameError: name 'train' is not defined

In [28]:
data = pd.concat([train, testA], axis=0, ignore_index=True)

## 数据预处理
- 可以看到很多变量不能直接训练，比如grade、subGrade、employmentLength、issueDate、earliesCreditLine，需要进行预处理


In [29]:
print(sorted(data['grade'].unique()))
print(sorted(data['subGrade'].unique()))

['A', 'B', 'C', 'D', 'E', 'F', 'G']
['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4', 'C5', 'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4', 'F5', 'G1', 'G2', 'G3', 'G4', 'G5']


In [30]:
data['employmentLength'].value_counts(dropna=False).sort_index()

employmentLength
1 year        65671
10+ years    328525
2 years       90565
3 years       80163
4 years       59818
5             11742
5 years       62645
6 years       46582
7 years       44230
8 years       45168
9 years       37866
< 1 year      80226
NaN           46799
Name: count, dtype: int64

- 首先对employmentLength进行转换到数值

In [31]:
data['employmentLength'].replace(to_replace='10+ years', value='10 years', inplace=True)
data['employmentLength'].replace('< 1 year', '0 years', inplace=True)

def employmentLength_to_int(s):
    if pd.isnull(s):
        return s
    else:
        return np.int8(s.split()[0])
    
data['employmentLength'] = data['employmentLength'].apply(employmentLength_to_int)

In [32]:
data['employmentLength'].value_counts(dropna=False).sort_index()

employmentLength
0.0      80226
1.0      65671
2.0      90565
3.0      80163
4.0      59818
5.0      74387
6.0      46582
7.0      44230
8.0      45168
9.0      37866
10.0    328525
NaN      46799
Name: count, dtype: int64

- 对earliesCreditLine进行预处理

In [33]:
data['earliesCreditLine'].sample(5)

776018    Sep-1988
280018    Jun-2009
623182    Nov-2005
617597    Mar-1994
385969    Sep-2000
Name: earliesCreditLine, dtype: object

In [34]:
# data['earliesCreditLine'] = data['earliesCreditLine'].apply(lambda s: int(s[-4:]))
# 修改处理方法
def process_credit_line(x):
    try:
        # 处理正常的年份格式（如 "Jun-2010"）
        return int(x[-4:])
    except:
        # 处理特殊情况（如 "v-74"）
        if 'v-' in x:
            return 1900 + int(x.split('-')[1])  # 假设v-74表示1974年
        else:
            return None  # 其他无法处理的情况返回空值

# 应用新的处理方法
data['earliesCreditLine'] = data['earliesCreditLine'].apply(process_credit_line)

In [35]:
data['earliesCreditLine'].describe()

count    817468.000000
mean       1997.618040
std          12.320809
min        1900.000000
25%        1994.000000
50%        2000.000000
75%        2004.000000
max        2015.000000
Name: earliesCreditLine, dtype: float64

In [36]:
data.head()

,id,loanAmnt,term,interestRate,installment,grade,subGrade,employmentTitle,employmentLength,homeOwnership,...,n5,n6,n7,n8,n9,n10,n11,n12,n13,n14
0,0,35000.0,5,19.52,917.97,E,E2,320.0,2.0,2,...,9.0,8.0,4.0,12.0,2.0,7.0,0.0,0.0,0.0,2.0
1,1,18000.0,5,18.49,461.90,D,D2,219843.0,5.0,0,...,NaN,NaN,NaN,NaN,NaN,13.0,NaN,NaN,NaN,NaN
2,2,12000.0,5,16.99,298.17,D,D3,31698.0,8.0,0,...,0.0,21.0,4.0,5.0,3.0,11.0,0.0,0.0,0.0,4.0
3,3,11000.0,3,7.26,340.96,A,A4,46854.0,10.0,1,...,16.0,4.0,7.0,21.0,6.0,9.0,0.0,0.0,0.0,1.0
4,4,3000.0,3,12.99,101.07,C,C2,54.0,NaN,1,...,4.0,9.0,10.0,15.0,7.0,12.0,0.0,0.0,0.0,4.0


- 类别特征处理

In [37]:
# 部分类别特征
cate_features = ['grade', 'subGrade', 'employmentTitle', 'homeOwnership', 'verificationStatus', 'purpose', 'postCode', 'regionCode', \
                 'applicationType', 'initialListStatus', 'title', 'policyCode']
for f in cate_features:
    print(f, '类型数：', data[f].nunique())

grade 类型数： 7
subGrade 类型数： 35
employmentTitle 类型数： 298101
homeOwnership 类型数： 6
verificationStatus 类型数： 3
purpose 类型数： 14
postCode 类型数： 935
regionCode 类型数： 51
applicationType 类型数： 2
initialListStatus 类型数： 2
title 类型数： 47903
policyCode 类型数： 1


In [38]:
# 类型数在2之上，又不是高维稀疏的
data = pd.get_dummies(data, columns=['grade', 'subGrade', 'homeOwnership', 'verificationStatus', 'purpose', 'regionCode'], drop_first=True)

In [40]:
# 高维类别特征需要进行转换
# for f in ['employmentTitle', 'postCode', 'title']:
#     data[f+'_cnts'] = data.groupby([f])['id'].transform('count')
#     data[f+'_rank'] = data.groupby([f])['id'].rank(ascending=False).astype(int)
#     del data[f]
for f in ['employmentTitle', 'postCode', 'title']:
    # 计算计数特征
    data[f+'_cnts'] = data.groupby([f])['id'].transform('count')
    
    # 计算排名特征，先填充NaN，再转换为整数
    data[f+'_rank'] = data.groupby([f])['id'].rank(ascending=False).fillna(0).astype(int)
    
    # 删除原始分类列
    del data[f]

## 训练数据/测试数据准备

In [41]:
features = [f for f in data.columns if f not in ['id','issueDate','isDefault']]

train = data[data.isDefault.notnull()].reset_index(drop=True)
test = data[data.isDefault.isnull()].reset_index(drop=True)

x_train = train[features]
x_test = test[features]

y_train = train['isDefault']

In [2]:
# 查看数据基本信息
print("训练数据形状:", x_train.shape)
print("测试数据形状:", x_test.shape)
print("标签数据形状:", y_train.shape)

# 查看数据类型
print("\n特征数据类型：")
print(x_train.dtypes)

# 查看前几行数据
print("\n处理后的数据示例：")
print(x_train.head())

# 查看统计信息
print("\n数据统计信息：")
print(x_train.describe())

NameError: name 'x_train' is not defined

## 模型训练
- 直接构建了一个函数，可以调用两种树模型，方便快捷

In [46]:
def cv_model(clf, train_x, train_y, test_x, clf_name):
    folds = 5
    seed = 2020
    kf = KFold(n_splits=folds, shuffle=True, random_state=seed)

    train = np.zeros(train_x.shape[0])
    test = np.zeros(test_x.shape[0])

    cv_scores = []

    for i, (train_index, valid_index) in enumerate(kf.split(train_x, train_y)):
        print('************************************ {} ************************************'.format(str(i+1)))
        trn_x, trn_y, val_x, val_y = train_x.iloc[train_index], train_y[train_index], train_x.iloc[valid_index], train_y[valid_index]

        if clf_name == "lgb":
            train_matrix = clf.Dataset(trn_x, label=trn_y)
            valid_matrix = clf.Dataset(val_x, label=val_y)

            params = {
                'boosting_type': 'gbdt',
                'objective': 'binary',
                'metric': 'auc',
                'min_child_weight': 5,
                'num_leaves': 2 ** 5,
                'lambda_l2': 10,
                'feature_fraction': 0.8,
                'bagging_fraction': 0.8,
                'bagging_freq': 4,
                'learning_rate': 0.1,
                'seed': 2020,
                'nthread': 28,
                'n_jobs':24,
                'silent': True,
                'verbose': -1,
            }

            # model = clf.train(params, train_matrix, 50000, valid_sets=[train_matrix, valid_matrix], verbose_eval=200,early_stopping_rounds=200)
            model = clf.train(params, train_matrix, num_boost_round=50000, valid_sets=[train_matrix, valid_matrix],
                            callbacks=[
                                lgb.early_stopping(stopping_rounds=200),
                                lgb.log_evaluation(period=200)]
            )
            val_pred = model.predict(val_x, num_iteration=model.best_iteration)
            test_pred = model.predict(test_x, num_iteration=model.best_iteration)
            
            # print(list(sorted(zip(features, model.feature_importance("gain")), key=lambda x: x[1], reverse=True))[:20])
                
        if clf_name == "xgb":
            train_matrix = clf.DMatrix(trn_x , label=trn_y)
            valid_matrix = clf.DMatrix(val_x , label=val_y)
            
            params = {'booster': 'gbtree',
                      'objective': 'binary:logistic',
                      'eval_metric': 'auc',
                      'gamma': 1,
                      'min_child_weight': 1.5,
                      'max_depth': 5,
                      'lambda': 10,
                      'subsample': 0.7,
                      'colsample_bytree': 0.7,
                      'colsample_bylevel': 0.7,
                      'eta': 0.04,
                      'tree_method': 'exact',
                      'seed': 2020,
                      'nthread': 36,
                      "silent": True,
                      }
            
            watchlist = [(train_matrix, 'train'),(valid_matrix, 'eval')]
            
            model = clf.train(params, train_matrix, num_boost_round=50000, evals=watchlist, verbose_eval=200, early_stopping_rounds=200)
            val_pred  = model.predict(valid_matrix, ntree_limit=model.best_ntree_limit)
            test_pred = model.predict(test_x , ntree_limit=model.best_ntree_limit)
                 
            
        train[valid_index] = val_pred
        test = test_pred / kf.n_splits
        cv_scores.append(roc_auc_score(val_y, val_pred))
        
        print(cv_scores)
       
    print("%s_scotrainre_list:" % clf_name, cv_scores)
    print("%s_score_mean:" % clf_name, np.mean(cv_scores))
    print("%s_score_std:" % clf_name, np.std(cv_scores))
    return train, test

In [47]:
def lgb_model(x_train, y_train, x_test):
    lgb_train, lgb_test = cv_model(lgb, x_train, y_train, x_test, "lgb")
    return lgb_train, lgb_test

def xgb_model(x_train, y_train, x_test):
    xgb_train, xgb_test = cv_model(xgb, x_train, y_train, x_test, "xgb")
    return xgb_train, xgb_test


In [48]:
lgb_train, lgb_test = lgb_model(x_train, y_train, x_test)

************************************ 1 ************************************
Training until validation scores don't improve for 200 rounds
[200]	training's auc: 0.742898	valid_1's auc: 0.730406
[400]	training's auc: 0.755553	valid_1's auc: 0.731185
[600]	training's auc: 0.766567	valid_1's auc: 0.731421
[800]	training's auc: 0.77656	valid_1's auc: 0.731297
Early stopping, best iteration is:
[658]	training's auc: 0.769561	valid_1's auc: 0.731571
[0.7315707699391983]
************************************ 2 ************************************
Training until validation scores don't improve for 200 rounds
[200]	training's auc: 0.743889	valid_1's auc: 0.726598
[400]	training's auc: 0.756346	valid_1's auc: 0.727829
[600]	training's auc: 0.767237	valid_1's auc: 0.728122
[800]	training's auc: 0.777257	valid_1's auc: 0.728164
Early stopping, best iteration is:
[700]	training's auc: 0.772432	valid_1's auc: 0.728318
[0.7315707699391983, 0.7283181812019169]
************************************ 3 ****

In [49]:
testA['isDefault'] = lgb_test
testA[['id','isDefault']].to_csv('test_sub.csv', index=False)